In [22]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
# pip install tensorflow==2.12.0  opencv-python mediapipe  matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [23]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [24]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [25]:
store_path = 'D:\\sem7\\CapstoneStuff'
dataset_dir = 'D:\\sem7\\CapstoneStuff\\dataset'
DATA_PATH = os.path.join(store_path,'MP_Data') 

# Actions that we try to detect
actions = np.array(['jumping', 'clapping','waving', 'namaste'])
# Thirty videos worth of data
no_sequences = 45
sequence_length = 65
# Folder start
start_folder = 30

In [26]:
def extract_keyp(results):
    pose = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*2)
    return pose

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [28]:
label_map = {label:num for num, label in enumerate(actions)}

In [29]:
label_map

{'jumping': 0, 'clapping': 1, 'waving': 2, 'namaste': 3}

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [31]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(65,66)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.load_weights("D:\sem7\CapstoneStuff\Pipe_Media_action_best.h5")

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 65, 64)            33536     
                                                                 
 lstm_4 (LSTM)               (None, 65, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 188,132
Trainable params: 188,132
Non-tr

In [35]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 

   

In [36]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [16]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.4
video_path="D:\\sem7\\CapstoneStuff\\test_videos\\clapping_T.mp4"
cap = cv2.VideoCapture(video_path)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    frame_counter = 0
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detections
        frame = cv2.resize(frame, (640, 640))
        image, results = mediapipe_detection(frame, holistic)
        frame_counter += 1
        if frame_counter % 2 == 0:  # Process every alternate frame
            continue
        # print(results)
        # Draw landmarks
        # frame = cv2.resize(frame, (640, 480))
        draw_styled_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keyp(results)
        sequence.append(keypoints)
        # sequence = sequence[-30:]
        if len(sequence) == 65:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sentence=actions[np.argmax(res)]
            print(sentence)
            predictions.append(np.argmax(res))    
        # res = model.predict(np.expand_dims(sequence, axis=0))[0]
        # sentence=actions[np.argmax(res)]
        # predictions.append(np.argmax(res)) 
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [38]:
print(actions)
print(res)

['jumping' 'clapping' 'waving' 'namaste']
[0.00127161 0.8026814  0.00117215 0.19487485]


In [25]:
print(sentence)

clapping


In [37]:
def predict_video(video_path):
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.4
    cap = cv2.VideoCapture(video_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        frame_counter = 0
        while cap.isOpened():
            # Read every alternate frame to speed up
            ret, frame = cap.read()
            frame_counter += 1
            if not ret or frame is None:
                break
            if frame_counter % 2 == 0:  # Process every alternate frame
                continue
            frame = cv2.resize(frame, (640, 720))
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            keypoints = extract_keyp(results)
            sequence.append(keypoints)
            if len(sequence) == 65:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                sentence=actions[np.argmax(res)]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                sequence = [] 
                break
            cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(20) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
        return sentence



In [38]:
predict_video("D:\\sem7\\CapstoneStuff\\test_videos\\Namaste_test.mp4")

1/1 [==============================] - 0s 455ms/step
namaste


'namaste'

In [20]:

# sequence = []
# sentence = []
# predictions = []
# threshold = 0.4
# video_path="D:\\sem7\\CapstoneStuff\\test_videos\\sparsh_clapping.mp4"
# cap = cv2.VideoCapture(str(video_path))
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#         frame_counter = 0
#         while cap.isOpened():
#             # Read every alternate frame to speed up
#             ret, frame = cap.read()
#             frame_counter += 1
#             if not ret or frame is None:
#                 break
#             if frame_counter % 2 == 0:  # Process every alternate frame
#                 continue
#             frame = cv2.resize(frame, (640, 720))
#             image, results = mediapipe_detection(frame, holistic)
#             draw_styled_landmarks(image, results)
#             keypoints = extract_keyp(results)
#             sequence.append(keypoints)
#             if len(sequence) == 65:
#                 res = model.predict(np.expand_dims(sequence, axis=0))[0]
#                 sentence=actions[np.argmax(res)]
#                 print(actions[np.argmax(res)])
#                 predictions.append(np.argmax(res))
#                 sequence = []  
#             cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
#             cv2.putText(image, ' '.join(sentence), (3, 30),
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#             # Show to screen
#             cv2.imshow('OpenCV Feed', image)    
#         cap.release()
#         cv2.destroyAllWindows()




1/1 [==============================] - 0s 40ms/step
clapping


In [42]:
# sequence = []
# sentence = []
# predictions = []
# threshold = 0.4
# video_path ="D:\\sem7\\CapstoneStuff\\test_videos\\Namaste_test.mp4"
# cap = cv2.VideoCapture(video_path)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#         frame_counter = 0
#         while cap.isOpened():
#             # Read every alternate frame to speed up
#             ret, frame = cap.read()
#             frame_counter += 1
#             if not ret or frame is None:
#                 break
#             if frame_counter % 2 == 0:  # Process every alternate frame
#                 continue
#             frame = cv2.resize(frame, (640, 720))
#             image, results = mediapipe_detection(frame, holistic)
#             draw_styled_landmarks(image, results)
#             keypoints = extract_keyp(results)
#             sequence.append(keypoints)
#             if len(sequence) == 65:
#                 res = model.predict(np.expand_dims(sequence, axis=0))[0]
#                 sentence=actions[np.argmax(res)]
#                 print(actions[np.argmax(res)])
#                 predictions.append(np.argmax(res))
#                 sequence = []  
#             cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
#             cv2.putText(image, ' '.join(sentence), (3, 30),
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#             # Show to screen
#             cv2.imshow('OpenCV Feed', image)    
#         cap.release()
#         cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
namaste
1/1 [==============================] - 0s 25ms/step
namaste
